In [1]:
import xarray as xr
import now
import marineHeatWaves
from dask.distributed import LocalCluster, Client

In [2]:
local_dir = "/g/data1a/e14/rm6294/dask-workers"
cluster = LocalCluster(processes=False, local_dir=local_dir)
client = Client(cluster)
client

Client Scheduler: inproc://10.0.64.22/626/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


In [3]:
cfg = 'present_vs_future.ini'
gridT = now.io.open_nemo_griddata_from_zarr(cfg, grid='T')
gridT

<xarray.Dataset>
Dimensions:          (simulation: 2, time_counter: 6940, x: 431, y: 289)
Coordinates:
    nav_lat          (y, x) float32 dask.array<shape=(289, 431), chunksize=(289, 431)>
    nav_lon          (y, x) float32 dask.array<shape=(289, 431), chunksize=(289, 431)>
  * time_counter     (time_counter) datetime64[ns] 1990-01-01T12:00:00 ... 2008-12-31T12:00:00
    time_maximum_1d  (time_counter) datetime64[ns] dask.array<shape=(6940,), chunksize=(6940,)>
    time_minimum_1d  (time_counter) datetime64[ns] dask.array<shape=(6940,), chunksize=(6940,)>
  * simulation       (simulation) <U7 'Present' 'Future'
Dimensions without coordinates: x, y
Data variables:
    20d              (simulation, time_counter, y, x) float32 dask.array<shape=(2, 6940, 289, 431), chunksize=(1, 250, 289, 431)>
    blt              (simulation, time_counter, y, x) float32 dask.array<shape=(2, 6940, 289, 431), chunksize=(1, 250, 289, 431)>
    depti            (simulation, time_counter, y, x) float32 dask

As we want surface temperatures, we try extracting the tos part of gridT. To confirm we run:

In [4]:
gridT.tos.attrs

OrderedDict([('interval_operation', '360s'),
             ('interval_write', '1d'),
             ('long_name', 'sea_surface_temperature'),
             ('online_operation', 'average'),
             ('units', 'degC')])

In [5]:
sst = gridT['tos']

In [6]:
sst

<xarray.DataArray 'tos' (simulation: 2, time_counter: 6940, y: 289, x: 431)>
dask.array<shape=(2, 6940, 289, 431), dtype=float32, chunksize=(1, 250, 289, 431)>
Coordinates:
    nav_lat          (y, x) float32 dask.array<shape=(289, 431), chunksize=(289, 431)>
    nav_lon          (y, x) float32 dask.array<shape=(289, 431), chunksize=(289, 431)>
  * time_counter     (time_counter) datetime64[ns] 1990-01-01T12:00:00 ... 2008-12-31T12:00:00
    time_maximum_1d  (time_counter) datetime64[ns] dask.array<shape=(6940,), chunksize=(6940,)>
    time_minimum_1d  (time_counter) datetime64[ns] dask.array<shape=(6940,), chunksize=(6940,)>
  * simulation       (simulation) <U7 'Present' 'Future'
Dimensions without coordinates: y, x
Attributes:
    interval_operation:  360s
    interval_write:      1d
    long_name:           sea_surface_temperature
    online_operation:    average
    units:               degC

In [7]:
sstPast = sst[0]
sstFut = sst[1]

In [23]:
sstPast['nav_lon'] = sstPast['nav_lon'] % 360
len(sstPast.isel(x = 100, y=100))

6940

In [59]:
    currSST = sstPast.isel(time_counter = i)
    currSST.plot(cmap='seismic', size = 10, x='nav_lon', y='nav_lat',vmin = 0, vmax = 37)
   
    plt.savefig(plot_dir + 'sstPlot_{:0>3d}.png'.format(i),bbox_inches='tight')
    plt.close('all')

#sstPast.isel(time_counter = 6000).plot(cmap='seismic', size = 10, x='nav_lon', y='nav_lat',vmin = 0, vmax = 37);

In [10]:
# define function to create movie
import os
import numpy as np
import matplotlib.pyplot as plt

def save_movie(image_filenames,movie_filename,framerate):
    os.system('ffmpeg -r ' + str(framerate) + ' -i ' + image_filenames + ' -vb 20M -vcodec mpeg4 -y ' + movie_filename)
    print('Created movie')

In [63]:
# create a directory for plots
plot_dir = "/g/data1a/e14/rm6294/sstPlots_v2"
lenT = 6940 # known previously

for i in range(19,100): 
    currSST = sstPast.isel(time_counter = i)
    currSST.plot(cmap='seismic', size = 10, x='nav_lon', y='nav_lat',vmin = 0, vmax = 37)
    
    plt.savefig(plot_dir + '/sstPlot_{:0>3d}.png'.format(i),bbox_inches='tight')
    plt.close('all')
    
    if i%50 == 0: 
        print('Saved figure',i)
        
print('Saved snapshots')

Saved figure 50
Saved snapshots


In [61]:
i

19